In [30]:
import os

from dotenv import load_dotenv

from parea import Parea

load_dotenv()

p = Parea(api_key=os.getenv("PAREA_API_KEY"))

dataset = p.get_collection(594)

dataset_rows = list(dataset.get_all_test_inputs_and_targets_tuple())

In [31]:
json.loads(dataset_rows[0][0])

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [32]:
dataset_rows[0][0]

{'model': 'gpt-3.5-turbo-0613',
 'tweet': "I love the new iPhone 13 Pro Max. It's the best phone I've ever had. I can't wait to get my hands on it. #iPhone13ProMax",
 'messages': '[\n  {\n    "content": "Tell me the sentiment of I love the new iPhone 13 Pro Max. It\'s the best phone I\'ve ever had. I can\'t wait to get my hands on it. #iPhone13ProMax. Only respond with \\"Positive\\", \\"Neutral\\" or \\"Negative\\"",\n    "role": "user"\n  }\n]',
 'provider': "OpenAIProvider('gpt-3.5-turbo-0613')",
 'functions': '[]',
 'model_params': '{\n  "model": "",\n  "temp": 0.5,\n  "top_p": 1,\n  "frequency_penalty": 0,\n  "presence_penalty": 0,\n  "max_length": 4024,\n  "response_format": null,\n  "safe_prompt": null\n}',
 'function_call': '{\n  "name": null\n}'}

In [35]:
from typing import Tuple, Dict, List
import json


def convert_dataset_to_ft_jsonl(rows: List[Tuple]) -> List[Dict]:
    jsonl_rows = []
    for inputs, target in rows:
        messages = json.loads(inputs["messages"])
        try:
            function_call = json.loads(target)
            if isinstance(function_call, List):
                function_call = function_call[0]
            if not "arguments" in function_call:
                # tool use format, need to convert
                function_call = function_call["function"]
            function_call["arguments"] = json.dumps(function_call["arguments"])
            assistant_response = {"role": "assistant", "function_call": function_call}
        except json.JSONDecodeError:
            assistant_response = {"role": "assistant", "content": target}
        messages.append(assistant_response)
        converted_row = {"messages": messages}
        if functions := inputs.get("functions", None):
            if loaded_functions := json.loads(functions):
                converted_row["functions"] = loaded_functions
        jsonl_rows.append(converted_row)
    return jsonl_rows

In [36]:
def write_jsonl(file_path, lines):
    """Create a .jsonl file and dump contents.
    file_path (unicode / Path): The path to the output file.
    lines (list): The JSON-serializable contents of each line.
    """
    data = [json.dumps(line) for line in lines]
    with open(file_path, "w", encoding="utf-8") as f:
        f.write("\n".join(data))


write_jsonl("finetuning.jsonl", convert_dataset_to_ft_jsonl(dataset_rows * 10))

In [10]:
function_call

{'name': 'solarFarm_potential',
 'arguments': {'coordinates': [43.653225, -79.383186],
  'month': 'December',
  'panelArea': 100000}}